# TensorFlow 설정하기

- tensorflow warning 비활화 

In [1]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [2]:
import tensorflow as tf

In [3]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.12.0


# 데이터세트 로드하기

In [4]:
mnist = tf.keras.datasets.mnist

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [7]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [8]:
y_train[0]

5

In [9]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# 머신 러닝 모델 빌드하기

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

- 상기 모델은 각 클래스에 대하여 logit 또는 log-odds 스코어 벡터를 반환

In [13]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.06044404, 0.06023994, 0.14078334, 0.07164061, 0.07796267,
        0.03039549, 0.08030642, 0.23345377, 0.16890459, 0.07586914]],
      dtype=float32)

- `tf.nn.softmax()`를 통하여 위의 logits 을 각 클래스에 대한 확률로 변환

In [14]:
tf.nn.softmax(predictions).numpy()

array([[0.09595247, 0.09593289, 0.10397934, 0.09703285, 0.09764824,
        0.09311213, 0.09787738, 0.11407575, 0.10694487, 0.09744402]],
      dtype=float32)

- `tf.nn.softmax` 함수를 네트워크의 마지막 레이어에 대한 활성화 함수로 베이킹
    - 모델 출력을 더 직접적으로 해석
    - softmax 출력을 사용할 경우 모든 모델에 대해 정확하고 수치적으로 안정적인 손실 계산을 제공하는 것이 불가능
    - 따라서 권장하지 않음
- `losses.SparseCategoricalCrossentropy`를 사용하여 logits의 벡터와 True 인덱스를 사용 -> 스칼라 손실을 반환

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

- 손실은 실제 클래스의 음의 로그 확률과 같음 -> 모델이 올바른 클래스를 확신하는 경우 손실은 0
- 예시 모델은 무작위에 가까운 확률(각 클래스에 대해 1/10)을 제공하므로 초기 손실은 `-tf.math.log(1/10) ~= 2.3`에 근접해야 함

In [16]:
loss_fn(y_train[:1], predictions).numpy()

2.3739507

- 상기 내용을 기반으로 모델 구성을 변경

In [17]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# 모델 훈련 및 평가하기

In [18]:
model.fit(x=x_train, y=y_train, epochs=5)

Epoch 1/5


/Users/rainmaker/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 [==============================] - 6s 3ms/step - loss: 0.2983 - accuracy: 0.9142
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1435 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1085 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0885 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0736 - accuracy: 0.9768


- 모델이 각 클래스에 대한 확률를 반환하도록 softmax 레이어를 추가

In [19]:
probability_model = tf.keras.models.Sequential([
    model, 
    tf.keras.layers.Softmax()
])

In [20]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.0853392 , 0.0853392 , 0.08533958, 0.0853552 , 0.0853392 ,
        0.0853392 , 0.0853392 , 0.23193067, 0.08533929, 0.08533936],
       [0.08536366, 0.08553751, 0.23154402, 0.08537009, 0.08536359,
        0.08536433, 0.08536367, 0.08536359, 0.08536599, 0.08536359],
       [0.08536512, 0.23152588, 0.08537886, 0.08537107, 0.08536829,
        0.08536846, 0.08539208, 0.08543203, 0.08543341, 0.08536479],
       [0.23193714, 0.08533879, 0.08534117, 0.08533887, 0.08533881,
        0.08533891, 0.08534983, 0.08533881, 0.08533878, 0.08533894],
       [0.08536221, 0.08535969, 0.08536039, 0.0853597 , 0.23160592,
        0.0853597 , 0.08536012, 0.08537832, 0.08535972, 0.08549428]],
      dtype=float32)>